In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Household Power Consumption
Additional explanations
- https://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+consumption
- https://machinelearningmastery.com/how-to-load-and-explore-household-electricity-usage-data/

Our Target: 'global_active_power' household global minute-averaged active power (in kilowatt). This represents the power consumed at current time.

In [5]:
df = pd.read_csv("data/household_power_consumption.txt", 
                 sep=";", 
                 low_memory=False, 
                 infer_datetime_format=True, 
                 parse_dates={'datetime':[0,1]}, 
                 index_col=['datetime']
                 )

In [6]:
len(df)

2075259

In [4]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [9]:
y = df.Global_active_power

In [10]:
y.head()

datetime
2006-12-16 17:24:00    4.216
2006-12-16 17:25:00    5.360
2006-12-16 17:26:00    5.374
2006-12-16 17:27:00    5.388
2006-12-16 17:28:00    3.666
Name: Global_active_power, dtype: object

In [13]:
# Missing values
sum(y == "?")

25979